<a href="https://colab.research.google.com/github/DiogofCabral/Data-processing/blob/main/spark/examples/06-write_partitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Write
- .write
- .format (parquet, csv, json)
- options
- spark.sql.sources.partitionOverwriteMode dynamic

# Write Mode
- overwrite - The overwrite mode is used to overwrite the existing file, alternatively, you can use SaveMode.Overwrite
- append - To add the data to the existing file, alternatively, you can use SaveMode.Append
- ignore - Ignores write operation when the file already exists, alternatively, you can use SaveMode.Ignore.
- errorifexists or error - This is a default option when the file already exists, it returns an error, alternatively, you can use SaveMode.ErrorIfExists.

# Partitioning
Process to organize the data into multiple chunks based on some criteria.
Partitions are organized in sub-folders.
Partitioning improves performance in Spark.

# Setting up PySpark

In [1]:
%pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').getOrCreate()

# Preparing data

In [3]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.0 MB/s eta 0:00:00


In [4]:
from faker import Faker
from datetime import datetime

fake = Faker()

users = []
for _ in range(50):
    user = {
        'date': fake.date_time_between_dates(datetime(2024, 5, 1), datetime(2024, 5, 5)),
        'name': fake.name(),
        'address': fake.address(),
        'email': fake.email(),
        'dob': fake.date_of_birth(),
        'phone': fake.phone_number()
    }
    users.append(user)

df = spark.createDataFrame(users)

df.show(10, False)


+---------------------------------------------------------+--------------------------+----------+-------------------------+-------------------+---------------------+
|address                                                  |date                      |dob       |email                    |name               |phone                |
+---------------------------------------------------------+--------------------------+----------+-------------------------+-------------------+---------------------+
|445 Sandy Lodge Apt. 016\nPort Carlosside, MN 30619      |2024-05-04 00:58:58.833235|1916-02-11|leah29@example.org       |Kristina Ross      |(450)911-1482        |
|611 Tracy Center\nMichaelmouth, TX 57404                 |2024-05-01 00:49:04.421481|1911-04-15|wtran@example.net        |Matthew Matthews   |(253)278-3661x732    |
|499 Jennifer Points\nSouth Kimberlyport, MP 11051        |2024-05-03 19:42:19.835249|1965-04-20|zrodriguez@example.net   |Elizabeth Hernandez|+1-222-382-1079x2517 |
|694

# Writing as PARQUET



In [5]:
# Writing as PARQUET with no partitions

path = "/content/write_partitioning/parquet_no_partitions"

df.write.mode("overwrite").format("parquet").save(path)

!ls /content/write_partitioning/parquet_no_partitions

spark.read.format("parquet").load(path).count()

part-00000-107acaba-31aa-4a38-95e2-4cb5754ebbbd-c000.snappy.parquet  _SUCCESS


50

In [6]:
# Writing as PARQUET with partitions
from pyspark.sql.functions import *

path = "/content/write_partitioning/parquet_with_partitions"

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic") # enable dynamic partition overwrite - only overwrites partitions that are coming in the dataframe

(df#.where("date_part = '20240503'")
 .write
 .mode("overwrite")                                               # overwrites the entire path with the new data
 .partitionBy("date_part")                                        # partition the data by column - creates sub-folders for each partition
 .format("parquet")                                               # format of output
 .save(path))                                                     # path

!ls /content/write_partitioning/parquet_with_partitions

spark.read.format("parquet").load(path).count()

'date_part=20240501'  'date_part=20240502'  'date_part=20240503'  'date_part=20240504'


50

In [7]:
# Checking single partition
spark.read.parquet("/content/write_partitioning/parquet_with_partitions/date_part=20240502").show()

+--------------------+--------------------+----------+--------------------+-------------------+-------------------+
|             address|                date|       dob|               email|               name|              phone|
+--------------------+--------------------+----------+--------------------+-------------------+-------------------+
|6436 Carpenter Hi...|2024-05-02 19:10:...|1917-09-07|robinsonsteve@exa...|       Pamela Adams|    +1-920-904-9289|
|PSC 2078, Box 239...|2024-05-02 16:14:...|1926-11-13|donnarussell@exam...|Christopher Gilmore|+1-392-706-9563x372|
|97918 Elizabeth B...|2024-05-02 00:29:...|1940-10-14| wwarren@example.net|   Ashley Jones DVM| 345.323.3589x87610|
|8087 Santos View ...|2024-05-02 01:21:...|1975-04-07|ravenballard@exam...|          Tara Hays|      (574)826-2009|
|34512 William Mou...|2024-05-02 05:06:...|2003-09-15|harmonlydia@examp...|        Daryl Miles|       323-474-0809|
|5708 Veronica Mil...|2024-05-02 02:53:...|1969-02-10| xtorres@example.n

# Writing as CSV

https://spark.apache.org/docs/3.5.1/sql-data-sources-csv.html

In [8]:
df.count()

50

In [9]:
path = "/content/write_partitioning/csv_no_partitioning/"

# write as csv
(df
  .write
  .format("csv")
  .mode("overwrite")
  .option("delimiter", "|")
  .option("header", True)
  .save(path))

# listing files in the folder
!ls /content/write_partitioning/csv_no_partitioning/

# read as csv
(spark
  .read
  .options(sep="|", multiLine=True, header=True)
  .csv(path)
  .count())

part-00000-43efe976-ad86-4cd2-87d7-e704bcdbcd96-c000.csv  _SUCCESS


50

# Writing as JSON

https://spark.apache.org/docs/3.5.1/sql-data-sources-json.html

In [11]:
path = "/content/write_partitioning/json_no_partitioning/"

# write as json
(df
.write
.mode("overwrite")
.format("json")
.save(path))

# listing files in the folder
!ls /content/write_partitioning/json_no_partitioning/

# read as json
(spark
  .read
  .json(path)
  .count())

part-00000-fef52904-23bb-4bda-b14d-665c015d27bb-c000.json  _SUCCESS


50

In [14]:
# reading json as text
spark.read.text(path).show(10, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                   |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"address":"445 Sandy Lodge Apt. 016\nPort Carlosside, MN 30619","date":"2024-05-04T00:58:58.833Z","dob":"1916-02-11","email":"leah29@example.org","name":"Kristina Ross","phone":"(450)911-1482","date_part":"20240504"}               |
|{"address":"611 Tracy Center\nMichaelmouth, TX 57404","date

In [15]:
# reading json as text
spark.read.json(path).show(10, False)

+---------------------------------------------------------+------------------------+---------+----------+-------------------------+-------------------+---------------------+
|address                                                  |date                    |date_part|dob       |email                    |name               |phone                |
+---------------------------------------------------------+------------------------+---------+----------+-------------------------+-------------------+---------------------+
|445 Sandy Lodge Apt. 016\nPort Carlosside, MN 30619      |2024-05-04T00:58:58.833Z|20240504 |1916-02-11|leah29@example.org       |Kristina Ross      |(450)911-1482        |
|611 Tracy Center\nMichaelmouth, TX 57404                 |2024-05-01T00:49:04.421Z|20240501 |1911-04-15|wtran@example.net        |Matthew Matthews   |(253)278-3661x732    |
|499 Jennifer Points\nSouth Kimberlyport, MP 11051        |2024-05-03T19:42:19.835Z|20240503 |1965-04-20|zrodriguez@example.net   

In [16]:
# partition json data + saveAsTable

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

# write as json
(df.write
  .partitionBy("date_part")
  .mode("overwrite")
  .format("json")
  .saveAsTable("tbl_json_part"))

# read as json
spark.table("tbl_json_part").count()

# read as json
spark.sql("show partitions tbl_json_part").show()

+------------------+
|         partition|
+------------------+
|date_part=20240501|
|date_part=20240502|
|date_part=20240503|
|date_part=20240504|
+------------------+



# Append Mode

In [17]:
# Writing as PARQUET with APPEND

path = "/content/write_partitioning/parquet_append"

df.write.mode("append").format("parquet").save(path)

!ls /content/write_partitioning/parquet_append

spark.read.format("parquet").load(path).count()

part-00000-de99dbf1-b9cf-4d5e-b57f-dd035cdc28ac-c000.snappy.parquet  _SUCCESS


50